
## Restaurant recommendation System


This is an extended version of my restaurnt suggester project <a href=''>(view the raw files here)</a> and <a href=''>(site over here) </a> 

<br>
In This project I have created a recommender function based of TF-IDF vectorizer and makes recommendations based of the content. I have also tried building a Tensorflow based recommender model which recommends the restaurnts. I have also tried K-means Algorithm and ALS Algorithm. I have described the steps that is involed in the code cells below. The diagram below depects the entire process of recommending restaurnts based of differrent algorithm.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark=SparkSession.builder.appName("suggester").getOrCreate()

In [2]:
df=spark.read.csv('cleaned_file.csv', header='true', inferSchema='false')

In [4]:
df.columns
df.printSchema()
df.show(5)

root
 |-- Unnamed: 0: string (nullable = true)
 |-- url: string (nullable = true)
 |-- address: string (nullable = true)
 |-- name: string (nullable = true)
 |-- online_order: string (nullable = true)
 |-- book_table: string (nullable = true)
 |-- rate: string (nullable = true)
 |-- votes: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- location: string (nullable = true)
 |-- rest_type: string (nullable = true)
 |-- dish_liked: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- cost: string (nullable = true)
 |-- reviews_list: string (nullable = true)
 |-- type: string (nullable = true)
 |-- cost_category: string (nullable = true)
 |-- bag_of_words: string (nullable = true)
 |-- lemmatized_reviews: string (nullable = true)

+---------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----

In [5]:
price_map = {
    'cheap-eats': ('cheap', 'inexpensive', 'low-price', 'low-cost', 'economical',
                   'economic', 'affordable'),
    'mid-range': ('moderate', 'fair', 'mid-price', 'reasonable', 'average'),
    'fine-dining': ('expensive', 'fancy', 'lavish')
}

In [35]:
from pyspark.sql.functions import udf
from pyspark.ml.feature import StopWordsRemover

def process_sentences(text):
    # Define a PySpark UDF (User Defined Function)
    @udf(returnType=F.StringType())
    def process_text(text):
        nlp = spacy.load("en_core_web_sm")  # Load a small English language model
        doc = nlp(text)

    processed_tokens = []
    for token in doc:
      # Lemmatize based on part-of-speech tags (similar to NLTK approach)
        if token.pos_ in ("VERB",):  # Only verbs
            lemma = token.lemma_.lower()  # Lowercase the lemma
        else:
            lemma = token.lemma_.lower()

      # Remove stop words and keep only alphanumeric characters
        if lemma not in spacy.lang.en.stop_words and lemma.isalnum():
            processed_tokens.append(lemma)

    # Join processed tokens with spaces and perform some cleanup (similar to NLTK)
    processed_text = " ".join(processed_tokens)
    processed_text = processed_text.replace("n't", " not")
    processed_text = processed_text.replace("'m", " am")
    processed_text = processed_text.replace("'s", " is")
    processed_text = processed_text.replace("'re", " are")
    processed_text = processed_text.replace("'ll", " will")
    processed_text = processed_text.replace("'ve", " have")
    processed_text = processed_text.replace("'d", " would")
    return processed_text

    return process_text(text)

# Create a StopWordsRemover object (optional, for additional stop word removal)
stop_words_remover = StopWordsRemover(inputCol="processed_text", outputCol="clean_text", stopWords=StopWordsRemover.loadStopWords("english"))

AttributeError: type object 'StopWordsRemover' has no attribute 'loadStopWords'

In [36]:
from pyspark.sql.functions import col, udf, lower, explode, split, array_contains
from pyspark.ml.feature import VectorAssembler


def preprocessing(input_text):
    # Fill missing values (consider replacing with your logic)
    df = df.fillna("unknown", ["location", "cuisines", "dish_liked", "rest_type", "reviews_list"])
    # Price mapping UDF (assuming 'cost' column exists)
    price_map_udf = udf(lambda x: next((v for k, v in price_map.items() if x.lower() in k), "unknown"), StringType())
    df = df.withColumn("price_category", price_map_udf(col("cost")))

    # Cost categorization UDF (assuming 'cost' column exists)
    cost_categorizer_udf = udf(lambda x: "cheap-eats restaurant" if x <= 200 
                               else ("med-range-restaurant" if x <= 800 else "fine-dining restaurant"), StringType())
    df = df.withColumn("cost_category", cost_categorizer_udf(col("cost")))

    # Define stop words and lemmatizer functions (UDFs)
    stop_words = set(nltk.corpus.stopwords.words("english"))

    def lemmatizer_udf(text):
        temp_sent = []
        words = split(lower(text), "\\W+")  # Split on non-word characters
        tags = nltk.pos_tag([w.rdd.first() for w in words.rdd])  # Get POS tags (assuming single partition)
        for i, word in enumerate(words.rdd.collect()):
            if tags[i][1] in ("VB", "VBD", "VBG", "VBN", "VBP", "VBZ"):  # Only verbs
                lemmatized = WordNetLemmatizer().lemmatize(word, pos="v")
            else:
                lemmatized = WordNetLemmatizer().lemmatize(word)
            if lemmatized not in stop_words and lemmatized.isalpha():
                temp_sent.append(lemmatized)
    full_sentence = " ".join(temp_sent)
    full_sentence = full_sentence.replace("n't", " not")
    # ... other replacements (same as original code)
    return full_sentence

    lemmatizer_udf = udf(lemmatizer_udf, StringType())

    # Create a new column with processed reviews using lemmatizer UDF
    df = df.withColumn("processed_reviews", lemmatizer_udf(explode(split(col("reviews_list"), "\\|"))))

    # Combine relevant columns into a bag-of-words feature
    assembler = VectorAssembler(inputCols=["rest_type", "processed_reviews"], outputCol="bag_of_words")


### Recommends restaurants based on name and lemmatized reviews (CBF)
<br>
Args:
<br>
- user_name: Name of the restaurant for which to recommend similar ones 
<br>
- data: Spark DataFrame containing restaurant data
<br>
- lemmatized_column: Column containing lemmatized reviews
<br>
- k: Number of recommendations to return
<br>
Returns:
- list: List of recommended restaurant names (excluding the user's restaurant)

In [18]:
from pyspark.sql.functions import col, udf, explode, split, array_contains,split
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.types import FloatType
from pyspark.ml import Pipeline


#Calculates cosine similarity between two sparse vectors
def cosine_similarity(vec1, vec2):
    return float(vec1.dot(vec2)) / (vec1.norm() * vec2.norm())
cosine_similarity_udf = udf(cosine_similarity, FloatType())


df = df.withColumn("review_list", split(col("lemmatized_reviews"), "\\|"))

def recommend_restaurants(user_name, data, lemmatized_column='lemmatized_reviews', k=10):
    # HashingTF for feature extraction
    hashingTF = HashingTF(inputCol=lemmatized_column, outputCol="features")

    # IDF for weighting terms
    idf = IDF(inputCol="features", outputCol="weighted_features")

    # Feature engineering pipeline
    assembler = Pipeline(stages=[hashingTF, idf])
    data_with_features = assembler.fit(data).transform(data)

    # Get user's review as a list of words
    user_review = data.filter(col("name") == user_name).select(lemmatized_column).rdd.flatMap(lambda x: x).first()

    # Convert user review to a sparse vector
    user_review_vector = Vectors.sparse(len(data.select(lemmatized_column).first()[0]), user_review)

    # Find most similar restaurants (excluding user's restaurant)
    top_k_similar = data_with_features.withColumn("similarity", cosine_similarity_udf(user_review_vector, col("weighted_features"))).filter(col("name") != user_name).sort(col('similarity').desc()).limit(k)  # Replace "id" with your restaurant ID column

    # Extract recommended restaurant names
    recommended_restaurants = top_k_similar.select("name").rdd.flatMap(lambda x: x).collect()

    return recommended_restaurants



In [28]:
from pyspark.sql import functions as F

def recommend(input_text):
    """
    Recommends restaurants based on user input using PySpark.

    Args:
      input_text: User's search query.

    Returns:
      str: HTML table containing top 10 recommended restaurants.
    """

    # Preprocess user input text
    processed_text = preprocessing(input_text)

    # Filter data based on extracted criteria
    filtered_data = df.where(
      F.col("location").isin(F.explode(F.split(processed_text, " "))).filter(F.col("location").isNotNull()) &
      F.col("cuisines").isin(F.explode(F.split(processed_text, " "))).filter(F.col("cuisines").isNotNull()) &
      F.col("dish_liked").isin(F.explode(F.split(processed_text, " "))).filter(F.col("dish_liked").isNotNull()) &
      F.col("rest_type").isin(F.explode(F.split(processed_text, " "))).filter(F.col("rest_type").isNotNull())
    )

    # Filter data based on price (assuming price_map is a dictionary)
    if price_map:
        for key, value in price_map.items():
            if any(v in processed_text for v in value):
                filtered_data = filtered_data.filter(F.col("price") == key)
                break

    # TF-IDF vectorization
    from pyspark.ml.feature import HashingTF, IDF

    hashingTF = HashingTF(inputCol="bag_of_words", outputCol="features")
    idf = IDF(inputCol="features", outputCol="tfidf_features")

    featurizedData = hashingTF.transform(filtered_data)
    idfModel = idf.fit(featurizedData)
    rescaledData = idfModel.transform(featurizedData)

    # Calculate cosine similarities
    input_vector = Vectors.dense(rescaledData.select("tfidf_features").first().toArray())
    cosine_similarities = rescaledData.select(
      col("name"), col("location"), col("cost"), col("cuisines"), col("reviews_list"),
      F.dot(col("tfidf_features"), input_vector) / (F.vector_norm(col("tfidf_features")) * F.vector_norm(input_vector))
    )

    # Sort and return recommendations
    recommendations = cosine_similarities.sort(
      F.col("dot(tfidf_features, input_vector) / (vector_norm(tfidf_features) * vector_norm(input_vector))").desc()
    ).select("name", "location", "cost", "cuisines", "reviews_list").limit(10).toHtml(index=False)

    return recommendations


In [29]:
user_name = "Pizza Palace"  # Replace with the restaurant name for recommendations
recommendations = recommend(user_name)

print(f"Recommendations for {user_name}:")
for restaurant in recommendations:
    print(restaurant)

TypeError: preprocessing() takes 0 positional arguments but 1 was given

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, lower, collect_list, lit
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.linalg import Vectors

In [ ]:
def recommend(input_text):
    """Recommends restaurants based on user input using PySpark."""

    # Lowercase input text
    input_text = lower(input_text)

    # Filter data based on extracted criteria
    data = df.filter(
        (col("location").isin(extract_list(input_text, "location", df))) &
        (col("cuisines").isin(extract_list(input_text, "cuisines", df))) &
        (col("dish_liked").isin(extract_list(input_text, "dish_liked", df))) &
        (col("rest_type").isin(extract_list(input_text, "rest_type", df))) &
        #(col("price") == extract_price(input_text, price_map))
    )

    # Process user description text input
    processed_input = process_sentences(input_text)

    # TF-IDF vectorization
    hashingTF = HashingTF(inputCol="bag_of_words", outputCol="features")
    featurizedData = hashingTF.transform(data)
    idf = IDF(inputCol="features", outputCol="tfidf_features")
    idfModel = idf.fit(featurizedData)
    rescaledData = idfModel.transform(featurizedData)

    # Calculate cosine similarities
    input_vector = Vectors.dense(rescaledData.select("tfidf_features").first().toArray())
    cosine_similarities = rescaledData.select(col("name"), col("location"), col("cost"), col("cuisines"), col("reviews_list"), dot(col("tfidf_features"), input_vector) / (vector_norm(col("tfidf_features")) * vector_norm(input_vector)))

    # Sort and return recommendations
    recommendations = cosine_similarities.sort(col("dot(tfidf_features, input_vector) / (vector_norm(tfidf_features) * vector_norm(input_vector))").desc()) \
                                            .select("name", "location", "cost", "cuisines", "reviews_list") \
                                            .toPandas() \
                                            .to_html(index=False)
    return recommendations

# Helper functions for extracting lists and price
extract_list_udf = udf(extract_list, ArrayType(StringType()))
#extract_price_udf = udf(extract_price, StringType())



In [ ]:
df.columns

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS

# Assuming you have a SparkSession named 'spark' and a DataFrame named 'data'
# with user_id, restaurant_id, and rating columns

# Prepare user-item interaction matrix
user_item_matrix = df.groupBy("name", "restaurant_id").agg(
    sum("rating").alias("rating")
)

# Train the ALS model
als = ALS(rank=10, maxIter=10, regParam=0.1, userCol="user_id", itemCol="restaurant_id", ratingCol="rating")
model = als.fit(user_item_matrix)

# Get user profile vector for a specific user
user_id = 123  # Replace with desired user ID
user_vector = model.userFactors.filter(col("id") == user_id).select("features").first()

# Recommend top-K restaurants for the user
top_k_recs = model.recommendForUser(user_id, 10)  # Recommend 10 restaurants
recommendations = top_k_recs.join(data.select("restaurant_id", "name"), on="restaurant_id") \
                            .select("name", "rating") \
                            .orderBy(col("rating").desc()) \
                            .toPandas()

print(f"Top 10 Recommendations for User {user_id}:")
print(recommendations)



In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

# Assuming you have a SparkSession named 'spark' and a DataFrame named 'data'
# with 'latitude' and 'longitude' columns for restaurant locations

# Create a feature vector for latitude and longitude
assembler = VectorAssembler(inputCols=["latitude", "longitude"], outputCol="features")
data_with_features = assembler.transform(data)

# Define the number of clusters (k) based on your analysis
num_clusters = 4  # Adjust this value as needed

# Train the K-Means model
kmeans = KMeans(k=num_clusters, seed=1)  # Set a random seed for reproducibility
model = kmeans.fit(data_with_features)

# Predict cluster labels for each restaurant
predictions = model.transform(data_with_features)
restaurants_with_cluster = predictions.select("name", "latitude", "longitude", "prediction")  # Assuming a 'name' column

# Analyze the clusters (optional)
# You can explore the centroids (cluster centers) and visualize the clusters on a map
centroids = model.clusterCenters()
print("Cluster centers:")
centroids.foreach(lambda row: print(row))

print("Sample restaurants with assigned clusters:")
restaurants_with_cluster.show(truncate=False)


In [ ]:
from tensorflow.keras.layers import Embedding, Dot, Dense

# User and Restaurant Embedding Layers (dimensions to be chosen)
user_embedding = Embedding(num_users, embedding_dim)
restaurant_embedding = Embedding(num_restaurants, embedding_dim)

# Additional Embedding Layers for other features (if applicable)

# User-Restaurant Interaction Layer (e.g., dot product)
user_restaurant_interaction = Dot(axes=1)([user_embedding(user_id), restaurant_embedding(restaurant_id)])

# Optional: Add hidden layers for more complex interactions

# Prediction Layer
prediction = Dense(1, activation="sigmoid")(user_restaurant_interaction)

# Model definition
model = tf.keras.Model(inputs=[user_id, restaurant_id], outputs=prediction)

# Compile the model with optimizer and loss function
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model on user-restaurant interaction data using distributed training 
# with TensorFlow/Keras on Spark
model.fit(..., epochs=...)

# Use the trained model to generate recommendations ...


# Content based filtering

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode, split, array_contains, similarity
from pyspark.ml.linalg import Vectors, VectorUDT


def recommend_restaurants(user_name, data, lemmatized_column='lemmatized_reviews', k=10):
  """
  Recommends restaurants based on name and lemmatized reviews (CBF)

  Args:
      user_name: Name of the restaurant for which to recommend similar ones
      data: Spark DataFrame containing restaurant data
      lemmatized_column: Column containing lemmatized reviews
      k: Number of recommendations to return

  Returns:
      list: List of recommended restaurant names (excluding the user's restaurant)
  """

  # Get user's review vector as a sparse vector
  user_review = data.filter(col("name") == user_name).select(col(lemmatized_column)).rdd.flatMap(lambda x: x).first()
  user_review_vector = Vectors.sparse(len(data.select(lemmatized_column).first()[0]), user_review)

  # Define a UDF for cosine similarity calculation
  cosine_similarity_udf = udf(lambda review_vec: review_vec.arraySimilarities(user_review_vector)[0], VectorUDT())

  # Add a new column with cosine similarity to all restaurants
  data_with_similarity = data.withColumn('similarity', cosine_similarity_udf(col(lemmatized_column)))

  # Get top k most similar restaurants (excluding the user's restaurant)
  top_k_similar = data_with_similarity.filter(col("name") != user_name).sort(col('similarity').desc()).limit(k)
  recommended_restaurants = top_k_similar.select('name').rdd.flatMap(lambda x: x).collect()

  return recommended_restaurants


# Example usage (assuming 'restaurants.csv' is your data file)
spark = SparkSession.builder.appName("RestaurantRecommender").getOrCreate()
data = spark.read.csv("restaurants.csv")  # Replace with your data path

user_name = "Pizza Palace"  # Replace with the restaurant name for recommendations
recommendations = recommend_restaurants(user_name, data)

print(f"Recommendations for {user_name}:")
for restaurant in recommendations:
  print(restaurant)

spark.stop()


# K-means Algorithm

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans


def cluster_restaurants(data_path, features, k):
  """
  Clusters restaurants using KMeans algorithm in PySpark

  Args:
      data_path: Path to the CSV file containing restaurant data
      features: List of column names to use for clustering (e.g., ['cost', 'rate'])
      k: Number of clusters

  Returns:
      Spark DataFrame: DataFrame containing restaurants with a new column "cluster"
  """

  spark = SparkSession.builder.appName("RestaurantClustering").getOrCreate()

  # Read data from CSV file
  data = spark.read.csv(data_path)

  # Create a feature vector assembler
  assembler = VectorAssembler(inputCols=features, outputCol="features")
  data_with_features = assembler.transform(data)

  # KMeans model
  kmeans = KMeans(k=k, seed=1)  # Set random seed for reproducibility

  # Fit the model on the features
  model = kmeans.fit(data_with_features)

  # Predict cluster labels for each restaurant
  data_with_cluster = model.transform(data_with_features)

  # Return DataFrame with the new "cluster" column
  return data_with_cluster.select(*data.columns, col("prediction").alias("cluster"))


# Example usage
data_path = "restaurants.csv"
features = ["cost", "rate"]  # Replace with relevant features for clustering
k = 3  # Number of clusters

clustered_restaurants = cluster_restaurants(data_path, features, k)

# Print restaurants with their assigned clusters
clustered_restaurants.show()

spark.stop()


# Tensorflow based recommender

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode, split

def prepare_data(data_path):
  """
  Prepares data for TF-RS model using PySpark

  Args:
      data_path: Path to the CSV file containing restaurant data

  Returns:
      tuple: Tuple containing lists of restaurant names and lemmatized reviews
  """

  spark = SparkSession.builder.appName("RestaurantRecommender").getOrCreate()
  data = spark.read.csv(data_path)

  restaurants = data.select("name").rdd.flatMap(lambda x: x).collect()

  def lemmatizer_udf(text):
    # ... Implement your lemmatization logic here (same as previous examples)
    return processed_text

  lemmatizer_udf = udf(lemmatizer_udf, StringType())
  reviews = data.withColumn("lemmatized_reviews", lemmatizer_udf(explode(split(col("reviews_list"), "\\|")))) \
             .select("lemmatized_reviews").rdd.flatMap(lambda x: x).collect()

  spark.stop()
  return restaurants, reviews


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding

# Define hyperparameters (adjust as needed)
embedding_dim = 128
epochs = 10

# Get data prepared by PySpark
restaurants, reviews = prepare_data("restaurant_data.csv")

# Create feature layers
restaurant_names = tf.keras.layers.Input(shape=(1,), dtype="string")
reviews_input = tf.keras.layers.Input(shape=(None,), dtype="string")

# Embedding layers for restaurants and reviews
restaurant_embedding = Embedding(len(restaurants) + 1, embedding_dim)(restaurant_names)
review_embedding = Embedding(max(len(w) for w in reviews) + 1, embedding_dim)(reviews_input)

# Average review embeddings to represent a restaurant
average_review_embedding = tf.keras.layers.GlobalAveragePooling1D()(review_embedding)

# Dot product for similarity between restaurant and average review embedding
dot_product = tf.keras.layers.Dot(axes=1)([restaurant_embedding, average_review_embedding])

# Recommender model
model = tf.keras.Model(inputs=[restaurant_names, reviews_input], outputs=dot_product)

# Compile the model (adjust loss function as needed)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))  # Adjust loss function

# Assuming you have prepared training data (restaurant, review, rating) as NumPy arrays
# Replace with your training logic
restaurant_names_train, reviews_train, labels_train = ...

# Train the model on your training data
model.fit([restaurant_names_train, reviews_train], labels_train, epochs=epochs)


In [ ]:
def recommend_restaurants(user_review, k=5):
  """
  Recommends restaurants based on a user's review

  Args:
      user_review: User's review as a list of words (lemmatized)
      k: Number of recommendations to return

  Returns:


In [ ]:
# Assuming you have your data in a pandas DataFrame (replace with your data loading logic)
import pandas as pd

df = pd.read_csv("restaurant_data.csv")  # Replace with your data path
restaurants = df["name"].tolist()
reviews = df["lemmatized_reviews"].tolist()


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding

# Define embedding dimension
embedding_dim = 128

# Create feature layers
restaurant_names = tf.keras.layers.Input(shape=(1,), dtype="string")
reviews_input = tf.keras.layers.Input(shape=(None,), dtype="string")

# Embedding layers for restaurants and reviews
restaurant_embedding = Embedding(len(restaurants) + 1, embedding_dim)(restaurant_names)
review_embedding = Embedding(max(len(w) for w in reviews) + 1, embedding_dim)(reviews_input)

# Average review embeddings to represent a restaurant
average_review_embedding = tf.keras.layers.GlobalAveragePooling1D()(review_embedding)

# Dot product for similarity between restaurant and average review embedding
dot_product = tf.keras.layers.Dot(axes=1)([restaurant_embedding, average_review_embedding])

# Recommender model
model = tf.keras.Model(inputs=[restaurant_names, reviews_input], outputs=dot_product)

# Compile the model (adjust loss function as needed)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))  # Adjust loss function


In [ ]:
# Assuming you have labeled data for training (restaurant, review, rating)
# Replace with your training logic

# Train the model on your prepared training data
model.fit([restaurant_names_train, reviews_train], labels_train, epochs=10)


In [ ]:
# Function to recommend restaurants based on a user's review

def recommend_restaurants(user_review, k=5):
  """
  Recommends restaurants based on a user's review

  Args:
      user_review: User's review as a list of words (lemmatized)
      k: Number of recommendations to return

  Returns:
      list: List of top k recommended restaurant names
  """

  # Convert user review to a tensor
  user_review_tensor = tf.convert_to_tensor([user_review])

  # Predict similarity scores for all restaurants
  predictions = model.predict([tf.fill((1,), restaurants[0]), user_review_tensor])

  # Get top k restaurant indexes with highest similarity scores
  top_k_indexes = predictions.argsort()[0][-k:][::-1]  # Sort descending and pick top k

  # Return recommended restaurant names
  return [restaurants[i] for i in top_k_indexes]

# Example usage
user_review = ["great", "pizza", "service"]  # Replace with user's review
recommendations = recommend_restaurants(user_review)

print(f"Recommended restaurants for your review:")
for restaurant in recommendations:
  print(restaurant)
